# Distance Map Generation with STARLING

This notebook will cover the basic functionality in STARLING, specifically distance map generation and various things you can calculate from the Ensemble object.. 
**Before starting**, install STARLING locally. See https://github.com/idptools/starling/ for more information

- This notebook will cover the following:
    1. Generating distance maps
    2. Specifying the number of steps and conformations
    3. Changing ionic strength 
    4. Using the ``Ensemble`` object
    5. Saving .starling files and trajectories

***Note***: STARLING can only generate ensembles of sequences up to 380 residues!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starling

#NOTE: if this is the first time you are running STARLING, it will download the model for you!
# this can take a few minutes depending on your internet speed.

In [ ]:
# first we are going to set the sequence we want to analyze.
# we are going to analyze the N-terminal IDR of Homo sapiens CTCF, Uniprot ID 49711
ctcf='MEGDAVEAIVEESETFIKGKERKTYQRRREGGQEEDACHLPQNQTDGGEVVQDVNSSVQMVMMEQLDPTLLQMKTEVMEGTVAPEAEAAVDDTQIITLQVVNMEEQPINIGELQLVQVPVPVTVPVATTSVEELQGAYENEVSKEGLAESEPMICHTLPLPEGFQVVKVGANGEVETLEQGELPPQEDPSWQKDPDYQPPAKKTKKTKKSKLRYTEEGKDVDVSVYDFEEEQQEGLLSEVNAEKVVGNMKPPKPTKIKKKGVKKTFQCELCSYTCPRRSNLDRHMKSHTDERPHKCHLCGRAFRTVTLLRNHLNTHTGTRPHKCPDCDMAFVTSGELVRHRRYKHTHEKPFKCSMCDYASVEVSKLKRHIRSHTGERPFQCSLCSYASRDTYKLKRHMRTHSGEKPYECYICHARFTQSGTMKMHILQKHTENVAKFHCPHCDTVIARKSDLGVHLRKQHSYIEQGKKCRYCDAVFHERYALIQHQKSHKNEKRFKCDQCDYACRQERHMIMHKRTHTGEKPYACSHCDKTFRQKQLLDMHFKRYHDPNFVPAAFVCSKCGKTFTRRNTMARHADNCAGPDGVEGENGGETKKSKRGRKRKMRSKKEDSSDSENAEPDLDDNEDEEEPAVEIEPEPEPQPVTPAPPPAKKRRGRPPGRTNQPKQNQPTAIIQVEDQNTGAIENIIVEVKKEPDAEPAEGEEEEAQPAATDAPNGDLTPEMILSMMDR'
# we used metapredict V3 to define the region that is the IDR of CTCF
ctcf_idr = ctcf[:262]

In [ ]:
# now we are going to generate distance maps for the CTCF IDR
# NOTE: By default STARLING will return a dictionary. If you input a string, the
# dictionary key is 'sequence_1'. Alternatively, we can set return_single_ensemble=True.
ctcf_ensemble = starling.generate(ctcf_idr, return_single_ensemble=True)

### Changing the number of steps and conformations

In [ ]:
# Note, you can change the number of steps and conformations. 
# if you do fewer steps or conformations, it will be faster but less accurate.
ctcf_fewer_steps_conformations = starling.generate(ctcf_idr, steps=10, conformations=100, return_single_ensemble=True)


### Changing the ionic strength

In [ ]:
# 20 mM ionic strength is considered low, 150 mM is physiological, and 300 mM is high.
ctcf_low_ionic_strength = starling.generate(ctcf_idr, salt=20, return_single_ensemble=True)
ctcf_high_ionic_strength = starling.generate(ctcf_idr, salt=300, return_single_ensemble=True)

### Visualizing distance maps

In [ ]:
# now let's visualize the distance maps!
# NOTE: Differences might be hard to see due to the large range of distances in the distance maps
# we are going to have 3 plots, 20 mm, 150 mm, and 300 mm ionic strength
mean_distance_maps = ctcf_ensemble.distance_maps(return_mean=True)
mean_distance_maps_low_ionic_strength = ctcf_low_ionic_strength.distance_maps(return_mean=True)
mean_distance_maps_high_ionic_strength = ctcf_high_ionic_strength.distance_maps(return_mean=True)
# get the min and max to set the color scale
vmin = min(np.min(mean_distance_maps_low_ionic_strength), np.min(mean_distance_maps), np.min(mean_distance_maps_high_ionic_strength))
vmax = max(np.max(mean_distance_maps_low_ionic_strength), np.max(mean_distance_maps), np.max(mean_distance_maps_high_ionic_strength))  
fig, axs = plt.subplots(1, 3, figsize=(15, 4), sharey=True, sharex=True)
im1 = axs[0].imshow(mean_distance_maps_low_ionic_strength, cmap='viridis', vmin=vmin, vmax=vmax)
axs[0].set_title('20 mM Ionic Strength')
im2 = axs[1].imshow(mean_distance_maps, cmap='viridis', vmin=vmin, vmax=vmax)
axs[1].set_title('150 mM Ionic Strength')
im3 = axs[2].imshow(mean_distance_maps_high_ionic_strength, cmap='viridis', vmin=vmin, vmax=vmax)
axs[2].set_title('300 mM Ionic Strength')
fig.colorbar(im3, ax=axs, orientation='vertical', fraction=.15)
plt.suptitle('Mean Distance Maps of ctcf IDR at Different Ionic Strengths')
plt.show()

In [ ]:
# now lets plot the difference between 20mm and 150mm, and 300mm and 150mm
fig, axs = plt.subplots(1, 2, figsize=(10, 4), sharey=True, sharex=True)
diff_low = mean_distance_maps_low_ionic_strength - mean_distance_maps
diff_high = mean_distance_maps_high_ionic_strength - mean_distance_maps
# get vmin vmax using the differences
vmax = max(np.max(diff_low), np.max(diff_high))
vmin = min(np.min(diff_low), np.min(diff_high))
range_value = max(abs(vmin), abs(vmax))

# Make symmetric and divide by 2 to make it easier to visuzlze
vmin = -range_value / 2
vmax = range_value / 2 

# plot
im1 = axs[0].imshow(diff_low, cmap='bwr', vmin=vmin, vmax=vmax)
axs[0].set_title('20 mM - 150 mM')
im2 = axs[1].imshow(diff_high, cmap='bwr', vmin=vmin, vmax=vmax)
axs[1].set_title('300 mM - 150 mM')
fig.colorbar(im2, ax=axs, orientation='vertical')
plt.suptitle('Difference in Mean Distance Maps of ctcf IDR at Different Ionic Strengths')
plt.show()

## Computing values from the ensemble.

### Distances between specified residues

In [ ]:
# We can compute various values from the ensemble. 
# distances between residues i and j. 
# note: this is 0-indexed, so residue 10 is the 11th residue in the sequence.
dist_11_51 = ctcf_ensemble.rij(10, 50)
print(f'Average distance between residues 11 and 51: {np.mean(dist_11_51):.2f} Å')

### end-to-end distance

In [ ]:
end_to_end_distances = ctcf_ensemble.end_to_end_distance()
print(f'Average end-to-end distance: {np.mean(end_to_end_distances):.2f} Å')

### Computing and visualizing contact maps

In [ ]:
# set threshold
threshold = 12.0
# set range for map. Note: we are setting vmax low to make it easier to see low probability contacts
vmax = 0.1
# we can get the contact maps for a structural ensemble. Contacts are defined by a threshold value in angstroms.
contact_maps = ctcf_ensemble.contact_map(contact_thresh=12.0, return_mean=True)
# plot the contact map
plt.imshow(contact_maps, cmap='viridis', vmin=0, vmax=vmax)
# add label to colorbar
cbar = plt.colorbar()
cbar.set_label('Contact Probability')
plt.xlabel('Residue Index')
plt.ylabel('Residue Index')
plt.title(f'Contact Map (Threshold = {threshold} Å)\nContact Probability 0 to {vmax}')
plt.show()

### Computing radius of gyration

In [ ]:
all_radius_of_gyration = ctcf_ensemble.radius_of_gyration()
print(f'Average radius of gyration: {np.mean(all_radius_of_gyration):.2f} Å')

### Computing local radius of gyration

In [ ]:
# if you want to get the local radius of gyration, you can specify a start and end. 
# remember the values are 0-indexed, so start=0 and end=50 will give you the radius of gyration for residues 1-50.
local_rg = ctcf_ensemble.local_radius_of_gyration(start=0, end=50)
print(f'Average local radius of gyration (residues 1-50): {np.mean(local_rg):.2f} Å')


### Computing hydrodynamic radius

In [ ]:
hydrodynamic_radii = ctcf_ensemble.hydrodynamic_radius()
print(f'Average hydrodynamic radius: {np.mean(hydrodynamic_radii):.2f} Å')

### Saving and loading ensemble objects.

To save time, you can save the ensemble object and load it later instead of having to recalculate everything!

In [ ]:
# to save the ensemble object, use the save function.
ctcf_ensemble.save('ctcf_ensemble')

In [ ]:
# loading the ensemble object
loaded_ensemble = starling.load_ensemble('ctcf_ensemble.starling')
print(loaded_ensemble)

### Saving a trajectory

In [ ]:
# You can also save the trajectory if you'd like to view it elsewhere.
# By default this saves a .xtc file and a .pdb file.
ctcf_ensemble.save_trajectory('ctcf_ensemble')

# note, you can save the entire thing as a PDB, but this uses much more storage.
# to do so, uncomment the line below.
#ctcf_ensemble.save_trajectory('ctcf_ensemble', pdb_trajectory=True)